In [1]:
%pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.6/766.6 MB 28.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 24.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 36.7 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 28.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 32.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 32.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 32.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.1/150.1 MB 30.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━

In [1]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
import pandas as pd

In [2]:
aapl_datetime = pd.read_csv('aapl_datetime.csv')

In [24]:
aapl_datetime['date']=pd.to_datetime(aapl_datetime['date'])

In [3]:
# Use BERT embeddings to convert headlines into vector space
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(aapl_datetime['headline'])

# Perform KMeans clustering on embeddings
num_clusters = 2  # You can adjust this based on your data
clustering_model = KMeans(n_clusters=num_clusters, random_state=42)
clustering_model.fit(embeddings)

# Assign each headline to a cluster
aapl_datetime['cluster'] = clustering_model.labels_

# Remove duplicates within each cluster (based on dates)
aapl_datetime_BERT = aapl_datetime.drop_duplicates(subset=['cluster', 'date'], keep='first').reset_index(drop=True)
print(aapl_datetime_BERT)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

      Unnamed: 0.1  Unnamed: 0                 time  \
0                0           0  2025-03-03 18:30:36   
1                2           2  2025-03-03 17:44:25   
2               12          12  2025-02-28 17:23:32   
3               13          13  2025-02-28 17:00:29   
4               20          20  2025-02-27 20:58:56   
...            ...         ...                  ...   
5024         20190       20190  2011-06-11 07:36:28   
5025         20191       20191  2011-06-06 21:33:12   
5026         20192       20192  2011-06-05 19:56:02   
5027         20193       20193  2011-05-21 05:33:49   
5028         20194       20194  2011-05-19 09:53:02   

                                               headline stock        date  \
0     AI Daily: Analyst sees Apple, Alibaba partners...  aapl  2025-03-03   
1     Is a $100B TSMC U.S. Manufacturing Investment ...  aapl  2025-03-03   
2     Nvidia (NVDA) Stock Rebounds from Post-Earning...  aapl  2025-02-28   
3     Dell reports mixed Q4 res

In [4]:
aapl_datetime_BERT.to_csv('aapl_datetime_BERT.csv', index=False)

In [6]:
a=model.similarity(embeddings,embeddings)

In [27]:
to_remove = set()

# Compare within ±3-day windows
for i in range(len(aapl_datetime)):
    if i in to_remove:
        continue
    current_date = aapl_datetime.loc[i, 'date']

    # Get indices of nearby dates (±3 days)
    nearby_indices = aapl_datetime[(aapl_datetime['date'] >= current_date - pd.Timedelta(days=5)) & 
                        (aapl_datetime['date'] <= current_date + pd.Timedelta(days=5))].index

    for j in nearby_indices:
        if i != j and j not in to_remove:
            similarity_score = a[i,j]
            if similarity_score > 0.6:
                to_remove.add(j)

# Remove duplicates
aapl_datetime_cleaned = aapl_datetime.drop(list(to_remove)).reset_index(drop=True)

In [36]:
aapl_datetime_cleaned=aapl_datetime_cleaned[['time','headline','stock','exact_time']]

In [37]:
aapl_datetime_cleaned.to_csv('aapl_datetime_BERT2.csv')